#### 라이브러리

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import time
from sklearn.metrics import accuracy_score
import warnings
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
# 경고 메시지 무시
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

In [71]:
data = pd.read_csv('machine_learing_data/머신러닝_통합.csv')
data.head(5)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist,song,genre
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범,박재범 - Like I Do,R&B/Soul
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON,VVON - Rewind,R&B/Soul
2,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Westcoast Marriage,NaN,Austn,Austn - Westcoast Marriage,R&B/Soul
3,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Unlock,NaN,차다빈,차다빈 - Unlock,R&B/Soul
4,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Beautiful,NaN,basecamp,basecamp - Beautiful,R&B/Soul


In [72]:
# 필요한 열 제외
data = data.drop(columns=['curArtist', 'curTitle'])

# 결측치 처리
data = data.fillna('')

# 범주형 변수를 category 타입으로 변환
categorical_cols = ['curColor', 'curColor2', 'curColor3', 'curObject', 'curHead', 'curExplain', 'genre', 'song']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# 'song' 열을 Label Encoding
label_encoder = LabelEncoder()
data['song'] = label_encoder.fit_transform(data['song'])

# 데이터 분리
X = data.drop(columns=['song'])
y = data['song']

# 학습용 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 모양 확인
print(X_train.shape, y_train.shape)

# DMatrix 생성
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

(5244, 10) (5244,)


In [73]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5244, 10) (5244,) (1311, 10) (1311,)


#### XGBoost 모델의 특징
- 효율적인 구현: XGBoost는 gradient boosting 알고리즘을 효율적으로 구현한 라이브러리입니다. 이를 통해 빠른 학습 속도와 높은 예측 성능을 달성할 수 있습니다.
- 다양한 작업 지원: XGBoost는 분류, 회귀, 순위 예측 등 다양한 기계학습 작업을 지원합니다.
- 자동 특성 중요도 계산: XGBoost는 자동으로 각 특성의 중요도를 계산하여 제공합니다. 이를 통해 특성 선택 및 모델 해석에 활용할 수 있습니다.
- 병렬 처리 지원: XGBoost는 병렬 처리를 지원하여 대용량 데이터에 대한 학습이 가능합니다.
#### XGBoost 모델의 하이퍼파라미터
- max_depth: 트리의 최대 깊이를 제한하는 하이퍼파라미터입니다. 과적합을 방지하기 위해 적절한 값을 선택해야 합니다.
- learning_rate: 각 트리에서 업데이트되는 가중치의 크기를 조절하는 하이퍼파라미터입니다. 작은 값을 사용하면 과적합을 방지할 수 있습니다.
- n_estimators: 생성할 트리의 개수를 지정하는 하이퍼파라미터입니다. 적절한 값을 선택하여 모델의 성능을 향상시킬 수 있습니다.
- min_child_weight: 리프 노드에서 필요한 최소 샘플 수를 지정하는 하이퍼파라미터입니다. 과적합을 방지하는 데 도움이 됩니다.

In [74]:
# 모델 파라미터 설정
params = {
    'objective': 'multi:softmax',
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_class': len(label_encoder.classes_)
}

# 학습 시간 측정
start_time = time.time()
model = xgb.train(params, dtrain, num_boost_round=100)
end_time = time.time()

# 학습 시간
training_time = end_time - start_time
print(f'학습 시간: {training_time} 초')

학습 시간: 172.99428129196167 초


In [75]:
# 모델 저장
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [76]:
# 모델 평가
y_pred = model.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.013729977116704805


In [77]:

# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred.astype(int))

# 예측 결과 출력
print("예측된 결과 (원래 값으로 변환):")
print(y_pred_original)

예측된 결과 (원래 값으로 변환):
['pH-1 - CITY LOVE' 'Gervs - Another Day' 'Valley - Like 1999' ...
 'Nieman - Brutally Honest' 'SOLE - RIDE' 'PATEKO - Sometimes']


In [78]:
# TF-IDF 벡터화
data['combined_text'] = data['curExplain'].astype(str) + ' ' + data['curHead'].astype(str) + ' ' + data['curObject'].astype(str)
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_text'])

# TF-IDF와 레이블 인코더 저장
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

#### 입력값 예측 함수

In [79]:
# 사용자 입력을 받아 예측하는 함수
def recommend_songs(tfidf_vectorizer, tfidf_matrix, label_encoder, input_keywords, input_genre, top_n=15):
    # 입력 키워드를 TF-IDF 벡터화
    input_text = ' '.join(input_keywords)
    input_vector = tfidf_vectorizer.transform([input_text])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(input_vector, tfidf_matrix)
    
    # 입력된 장르와 일치하는 곡들만 선택
    genre_matches = data[data['genre'].str.contains(input_genre, case=False, na=False)]
    genre_indices = genre_matches.index
    
    # 유사도가 가장 높은 상위 n개 인덱스 찾기 (장르 일치하는 곡들 중)
    filtered_cosine_sim = cosine_sim[0, genre_indices]
    top_n_indices = genre_indices[np.argsort(filtered_cosine_sim)[-top_n:][::-1]]
    
    # 가장 유사한 노래 추천
    recommended_songs = data.iloc[top_n_indices]['song']
    recommended_songs = label_encoder.inverse_transform(recommended_songs)
    
    return recommended_songs

In [80]:
# 저장된 TF-IDF 벡터화 및 레이블 인코더 불러오기
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

# 저장된 모델 불러오기
with open('xgboost_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


In [81]:
# 사용자 입력 예제
input_keywords = ['신나는', '드라이브', '바다']
input_genre = 'kpop'

# 예측 수행
recommended_songs = recommend_songs(tfidf_vectorizer, tfidf_matrix, label_encoder, input_keywords, input_genre, top_n=15)
print("Recommended songs:")
for song in recommended_songs:
    print(song)

Recommended songs:
